In [1]:
import pandas as pd
import numpy as np
import math


filename = 'Data_for_gchoice.csv';

GOV = pd.read_csv(filename)
GOV

,win,dcn,drn,diffc,diffr,lndiffc,index,n,cash,roy,oroy,ocash
0,0,0.558,0.180,52.65585,-0.033333,0.105675,0.296,1,525.0729,0.166667,0.200000,472.4170
1,1,0.442,0.812,-52.65585,0.033333,-0.105675,0.296,1,472.4170,0.200000,0.166667,525.0729
2,0,0.526,0.180,22.13177,-0.033333,0.043064,0.200,2,525.0729,0.166667,0.200000,502.9411
3,1,0.474,0.812,-22.13177,0.033333,-0.043064,0.200,2,502.9411,0.200000,0.166667,525.0729
4,0,0.458,0.418,-150.68920,0.000000,-0.082144,0.878,3,1760.1450,0.200000,0.200000,1910.8350
...,...,...,...,...,...,...,...,...,...,...,...,...
1441,1,0.310,0.788,-263.60470,0.030000,-0.345916,0.972,721,637.8293,0.250000,0.220000,901.4340
1442,0,0.318,0.418,-199.01900,0.000000,-0.320929,0.782,722,525.9387,0.230000,0.230000,724.9577
1443,1,0.680,0.418,199.01900,0.000000,0.320929,0.782,722,724.9577,0.230000,0.230000,525.9387
1444,0,0.386,0.708,-248.37060,0.017500,-0.193218,0.828,723,1165.2530,0.220000,0.202500,1413.6230


In [2]:
def nchoosek(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [3]:
def bernstein_bases_univariate(x, nx) :
    B = np.empty([len(x), nx+1])
    for p in range(nx + 1):
        B[:,p] = ( nchoosek(nx,p)* x ** p ) * ((1-x) ** (nx-p)) 
    return B

In [8]:
def bernstein_bases_trivariate(x, y, z, nx, ny, nz) :
    
    # sample size
    numS = len(x)

    Bx = bernstein_bases_univariate( x, nx )
    By = bernstein_bases_univariate( y, ny )
    Bz = bernstein_bases_univariate( z, nz )

    Bxym  = np.zeros((numS, nx+1, ny+1))
    Bxy   = np.zeros((numS, (nx+1)*(ny+1)))
    Bxyzm = np.zeros((numS, (nx+1)*(ny+1), nz+1))
    Bxyz  = np.zeros((numS, (nx+1)*(ny+1)*(nz+1)))

    for i in range(numS):
        Bxym[i,:,:] = np.outer(Bx[i,:], By[i,:])

    for i in range(numS):
        Bxy[i,:] = Bxym[i,:,:].flatten()
    
    for i in range(numS):
        Bxyzm[i,:,:] = np.outer(Bxy[i,:], Bz[i,:])

    for i in range(numS):
        Bxyz[i,:] = Bxyzm[i,:,:].flatten()


    return Bxyz

In [36]:
def gchoice_likelihood(beta,bases,win) :

    numS = bases.shape[0]
    win = win.reshape((numS, 1))

    P = bases @ beta.reshape((147, 1))
    likelihood = (P ** win) * ((1-P) ** (1-win));
    L = sum(np.log(likelihood))

    return L

In [38]:
x = GOV.drn.array
y = GOV.dcn.array
z = GOV.index.array
win = GOV.win.array

nx = 6
ny = 6
nz = 2

beta = np.arange(147) + 1

bases = bernstein_bases_trivariate(x, y, z, nx, ny, nz)

numS = bases.shape[0]
win = win.reshape((numS, 1))

P = bases * beta
likelihood = (P ** win) * ((1-P) ** (1-win));
L = sum(np.log(likelihood))

gchoice_likelihood(beta,bases,win)

<PandasArray>
[nan]
Length: 1, dtype: float64

In [35]:
P = bases @ beta.reshape((147, 1))
P


array([[  32.724     ],
       [ 112.268     ],
       [  36.148     ],
       ...,
       [2950.908     ],
       [2984.15599998],
       [2937.556     ]])

In [33]:
beta = np.arange(147)
beta.reshape((147, 1))

array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
      

147